In [1]:
import os
import subprocess
import time
import ssl
import webbrowser
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# ✅ Temporarily disable SSL verification
ssl._create_default_https_context = ssl._create_unverified_context

# ✅ Google API configuration
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = "client_secret.json"
TOKEN_FILE = "token.json"
YOUTUBE_MAX_DURATION = 43200  # 12 hours in seconds

# ✅ Timeouts
MERGE_TIMEOUT = 1200  # 20 minutes
CONVERT_TIMEOUT = 7200  # 2 hours

def get_authenticated_service():
    """
    Authenticate with YouTube API and return a service object.
    If re-authentication is required, open a browser before starting the process.
    """
    creds = None

    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                print("🔄 Token expired. Attempting to refresh...")
                creds.refresh(Request())
                print("✅ Token refreshed successfully!")
            except Exception as e:
                print(f"⚠️ Token refresh failed: {e}\n🌐 Opening browser for re-authentication...")
                creds = None

        if creds is None:
            print("🌐 Opening browser for authentication...")
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(TOKEN_FILE, "w") as token_file:
            token_file.write(creds.to_json())

    return build("youtube", "v3", credentials=creds)

# ✅ Authenticate before starting the process
youtube_service = get_authenticated_service()
print("✅ Authentication successful! Now processing videos...")

def get_video_duration(video_path):
    """Returns the duration of a video in seconds."""
    try:
        result = subprocess.run(
            ["ffprobe", "-v", "error", "-show_entries", "format=duration",
             "-of", "default=noprint_wrappers=1:nokey=1", video_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        return float(result.stdout.strip())
    except Exception:
        print(f"⚠️ Warning: Could not read duration for {video_path}. Skipping file...")
        return None

def convert_videos(folder_path, video_files):
    """Converts videos to a uniform format (H.264, AAC, 720p, 30 FPS)."""
    converted_folder = os.path.join(folder_path, "converted_videos")
    os.makedirs(converted_folder, exist_ok=True)
    converted_files = []

    print(f"🔄 Starting video conversion... (Timeout: {CONVERT_TIMEOUT // 3600} hours)")
    
    start_time = time.time()
    for video in video_files:
        converted_video = os.path.join(converted_folder, os.path.basename(video))

        try:
            subprocess.run(
                ["ffmpeg", "-i", video, "-vf", "scale=1280:720", "-r", "30", 
                 "-c:v", "libx264", "-preset", "fast", "-crf", "23", "-c:a", "aac", "-b:a", "128k",
                 "-strict", "experimental", converted_video],
                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=CONVERT_TIMEOUT
            )
        except subprocess.TimeoutExpired:
            print(f"⏳ Conversion **timed out**! Skipping this subfolder...")
            return []

        converted_files.append(converted_video)

        if time.time() - start_time > CONVERT_TIMEOUT:
            print("⏳ Total conversion exceeded allowed time. Aborting conversion...")
            return []

    print("✅ Video conversion complete!")
    return converted_files

def try_merge_videos(folder_path, video_files, expected_duration, merge_attempt):
    """Merges videos with forced re-encoding to avoid codec mismatches."""
    file_list_path = os.path.join(folder_path, "file_list.txt")
    merged_video_path = os.path.join(folder_path, "merged_video.mp4")

    with open(file_list_path, "w", encoding="utf-8") as f:
        for video_file in video_files:
            f.write(f"file '{video_file}'\n")

    print(f"🔗 Attempt {merge_attempt}: Merging videos... (Timeout: {MERGE_TIMEOUT // 60} min)")

    try:
        subprocess.run(
            ["ffmpeg", "-f", "concat", "-safe", "0", "-i", file_list_path, 
             "-c:v", "libx264", "-preset", "fast", "-crf", "23", "-c:a", "aac", "-b:a", "128k", 
             "-strict", "experimental", merged_video_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=MERGE_TIMEOUT
        )
    except subprocess.TimeoutExpired:
        print(f"⏳ Merge attempt **timed out**! Skipping...")
        return None

    merged_duration = get_video_duration(merged_video_path)
    if merged_duration and abs(merged_duration - expected_duration) <= 5:
        print(f"✅ Merge successful! Duration: {merged_duration:.2f} sec")
        return merged_video_path
    else:
        print(f"❌ Merge failed. Removing incorrect file...")
        os.remove(merged_video_path)
        return None

def upload_to_youtube(video_path, title, description, folder_path):
    """Uploads a video to YouTube and saves the video URL in 'youtube_link.txt'."""
    print(f"📤 Uploading: {title}...")

    youtube = get_authenticated_service()
    request = youtube.videos().insert(
        part="snippet,status",
        body={"snippet": {"title": title, "description": description, "categoryId": "27"}, 
              "status": {"privacyStatus": "unlisted"}},
        media_body=MediaFileUpload(video_path, chunksize=-1, resumable=True)
    )

    for attempt in range(3):
        try:
            _, response = request.next_chunk()
            video_url = f"https://www.youtube.com/watch?v={response['id']}"
            with open(os.path.join(folder_path, "youtube_link.txt"), "w") as f:
                f.write(video_url + "\n")
            print(f"✅ Upload complete! Video URL: {video_url}")
            return video_url
        except Exception as e:
            print(f"⚠️ Upload failed (Attempt {attempt + 1}/3). Retrying in 60 sec...")
            time.sleep(60)

    print("❌ Upload failed after 3 attempts.")
    return None

if __name__ == "__main__":
    parent_folder = input("📂 Enter the parent folder containing all video subfolders: ").strip()

    for subfolder in sorted(os.listdir(parent_folder)):
        subfolder_path = os.path.join(parent_folder, subfolder)

        if os.path.isdir(subfolder_path):
            print(f"🚀 Processing: {subfolder_path}")
            video_files = sorted([os.path.join(subfolder_path, f) for f in os.listdir(subfolder_path) if f.endswith('.mp4')])
            merged_video_path, description, _ = try_merge_videos(subfolder_path, video_files, sum(get_video_duration(f) or 0 for f in video_files), 1)
            if merged_video_path:
                upload_to_youtube(merged_video_path, subfolder, description, subfolder_path)

print("✅ All subfolders processed!")


✅ Authentication successful! Now processing videos...
🚀 Processing: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\Photoshop\Lynda\Learning Photoshop Automation
🔗 Attempt 1: Merging videos... (Timeout: 20 min)
⚠️ Warning: Could not read duration for H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\Photoshop\Lynda\Learning Photoshop Automation\merged_video.mp4. Skipping file...
❌ Merge failed. Removing incorrect file...


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'H:\\Projects Control (PC)\\10 Backup\\05 Tutorials\\Adobe\\Photoshop\\Lynda\\Learning Photoshop Automation\\merged_video.mp4'